In [2]:
import os
import numpy as np
import pandas as pd
import cv2
import gc
from tqdm import tqdm
from glob import glob

## Collect all data and label them

In [3]:
dirs = os.listdir('data')
images_path = []
labels = []
for folder in dirs:
    path = glob('./data/{}/*.png'.format(folder))
    label = ['{}'.format(folder)]*len(path)
    images_path.extend(path)
    labels.extend(label)

In [4]:
#images_path
print(dirs)

['data_preprocess.npz', 'mask', 'no mask']


# Face Detection
Face Detection model with Caffe implementation

In [5]:
img_path = images_path[140]
img = cv2.imread(img_path)

In [6]:
cv2.imshow('original', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [7]:
#load face detection model
face_detection_model = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt.txt',
                                                './models/res10_300x300_ssd_iter_140000_fp16.caffemodel')

In [8]:
def face_detection_dnn(img):
    #rgb mean subtraction image
    image = img.copy()
    h,w = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,117,123), swapRB = True) #rgb value and model resize
    #detection
    face_detection_model.setInput(blob)
    detections = face_detection_model.forward()
    for i in range(0,detections.shape[2]):
        confidence = detections[0,0,i,2] # confidence score
        print(confidence)
        if(confidence > 0.5):
            box = detections[0,0,i,3:7]*np.array([w,h,w,h]) #bounding box
            box = box.astype(int)
            pt1 = (box[0], box[1])
            pt2 = (box[2], box[3])
            cv2.rectangle(img, pt1, pt2,(255,0,0),1)
            #crop the face
            roi = image[box[1]:box[3],box[0]:box[2]]
            print()
            
            return roi
    return None

In [9]:
img_roi = face_detection_dnn(img)
cv2.imshow('roi',img_roi)
cv2.waitKey(0)
cv2.destroyAllWindows

0.92438



<function destroyAllWindows>

# RGB of Image

In [10]:
def datapreprocess(img):
    face = face_detection_dnn(img)
    if face is not None:
        blob = cv2.dnn.blobFromImage(face,1,(100,100),(104,117,123),swapRB = True)
        blob_squeeze = np.squeeze(blob).T
        blob_rotate = cv2.rotate(blob_squeeze,cv2.ROTATE_90_CLOCKWISE)
        blob_flip = cv2.flip(blob_rotate,1)
        print(blob_flip)
        #remove negative values and normalize
        img_norm = np.maximum(blob_flip,0)/blob_flip.max()
    
        return img_norm
    else: 
        return None
    
    

In [11]:
img_blob = datapreprocess(img)
cv2.imshow('blob',img_blob)
cv2.waitKey(0)
cv2.destroyAllWindows

0.5512009

[[[ -69.  -90. -104.]
  [ -68.  -89. -103.]
  [ -68.  -89. -103.]
  ...
  [ -78.  -97. -112.]
  [ -81.  -99. -112.]
  [ -82.  -99. -112.]]

 [[ -65.  -88. -102.]
  [ -64.  -87. -102.]
  [ -65.  -87. -102.]
  ...
  [ -77.  -95. -110.]
  [ -79.  -97. -110.]
  [ -81.  -98. -110.]]

 [[ -62.  -86. -101.]
  [ -62.  -85. -101.]
  [ -63.  -86. -100.]
  ...
  [ -75.  -94. -108.]
  [ -78.  -95. -109.]
  [ -80.  -97. -109.]]

 ...

 [[ -76.  -95. -103.]
  [ -73.  -92. -101.]
  [ -70.  -89.  -98.]
  ...
  [  20.   13.   10.]
  [  21.   13.   10.]
  [  21.   12.   10.]]

 [[ -79.  -97. -105.]
  [ -79.  -98. -106.]
  [ -78.  -96. -105.]
  ...
  [  17.    9.    6.]
  [  18.   10.    7.]
  [  19.   10.    7.]]

 [[ -75.  -93. -101.]
  [ -80.  -98. -106.]
  [ -81.  -98. -107.]
  ...
  [  12.    4.    1.]
  [  13.    5.    2.]
  [  14.    5.    2.]]]


<function destroyAllWindows>

In [10]:
data_img = []
label_img = []
i = 0
for path,label in tqdm(zip(images_path, labels), desc = 'preprocessing'):
    img = cv2.imread(path)
    process_img = datapreprocess(img)
    if process_img is not None:
        data_img.append(process_img)
        label_img.append(label)
        
    i += 1
    if i%100 == 0:
        gc.collect()

preprocessing: 4it [00:00, 18.15it/s]

0.99933136

0.5520837

0.9731012

0.9519349



preprocessing: 7it [00:00, 20.01it/s]

0.562616

0.28785396
0.13686526
0.11300726
0.10237725
0.08864273
0.08621167
0.082998805
0.08266548
0.08153399
0.08111103
0.0809627
0.078346774
0.077826045
0.0776821
0.07740159
0.077238016
0.07696863
0.07668061
0.07640402
0.075910985
0.07576618
0.07571407
0.07508206
0.074895926
0.07485516
0.07482097
0.07462546
0.07462036
0.07453894
0.074330725
0.07391067
0.07379685
0.07371197
0.0736323
0.07354159
0.0735072
0.073375635
0.0730115
0.0727049
0.07263754
0.07260607
0.072489455
0.072421014
0.07224627
0.07219271
0.07214821
0.07208717
0.07201347
0.07194698
0.07179713
0.071706966
0.07163988
0.07158156
0.07154101
0.07153489
0.07138607
0.07137251
0.07116161
0.07102053
0.07090787
0.0708842
0.07085937
0.07081937
0.070814036
0.070710726
0.070632525
0.070591874
0.07057649
0.0703263
0.07026746
0.07019304
0.0701892
0.07018463
0.07016625
0.07004442
0.07002048
0.070009395
0.06997358
0.06982775
0.069708325
0.06966777
0.06964298
0.06958279
0.069515735
0.06949696
0.06935632
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

preprocessing: 13it [00:00, 21.12it/s]

...

0.9995432

0.9997056

0.999918

0.9996338

0.9998983

0.9999459



preprocessing: 5972it [03:36, 26.05it/s]

0.99994874

0.9998759

0.9999509

0.9999763

0.9999558

0.9986676



preprocessing: 5976it [03:36, 27.55it/s]

0.99996996

0.9999628



In [16]:
X = np.array(data_img)
y = np.array(label_img)

In [17]:
X.shape

(5620, 100, 100, 3)

In [ ]:
np.savez('./data/data_preprocess.npz',X,y)

In [ ]:
def datapreprocess(img):
    face = face_detection_dnn(img)
    if face is not None:
        blob = cv2.dnn.blobFromImage(face,1,(100,100),(104,117,123),swapRB = True)
        blob_squeeze = np.squeeze(blob).T
        blob_rotate = cv2.rotate(blob_squeeze,cv2.ROTATE_90_CLOCKWISE)
        blob_flip = cv2.flip(blob_rotate,1)
        print(blob_flip)
        #remove negative values and normalize
        img_norm = np.maximum(blob_flip,0)/blob_flip.max()
    
        return img_norm
    else: 
        return None
    
    